### XGBoost Built in Algorithm training for Sagemaker Endpoint

In [1]:
import numpy as np
import pandas as pd

import boto3
import re

import sagemaker
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Bucket name is hidden using environment variable

sh-4.2$ cat .env<br>
BUCKET_NAME=my_ml_bucket_name

In [2]:
### Import s3 bucket name as environment variable

import os
env_vars = !cat ./.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [3]:
# Initializing s3 bucket and s3 locations

bucket_name = os.environ['BUCKET_NAME']

training_folder=r'bikerental/training/'
validation_folder=r'bikerental/validation/'
test_folder=r'bikerental/test/'

s3_model_output_location =r's3://{0}/bikerental/model'.format(bucket_name)
s3_training_file_location =r's3://{0}/{1}'.format(bucket_name,training_folder)
s3_validation_file_location = r's3://{0}/{1}'.format(bucket_name,validation_folder)
s3_test_file_location = r's3://{0}/{1}'.format(bucket_name,test_folder)

In [ ]:
print(s3_model_output_location)

In [ ]:
print(s3_training_file_location)
print(s3_validation_file_location)
print(s3_test_file_location)

In [6]:
# write files in s3 using boto3
# filename - filename
# bucket - bucket name
# Key - file location in s3(folder path)

def write_s3(filename, bucket, key):
    with open(filename,'rb') as f:
        boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)
        

In [9]:
# Writing all three Train, Validation and TEst files in s3

# uploading file with key can create subfolders 

write_s3('bike_train_rev3.csv', bucket_name,training_folder+'bike_test_rev3.csv')
write_s3('bike_validation_rev3.csv', bucket_name,validation_folder+'bike_validation_rev3.csv')
write_s3('bike_test_rev3.csv', bucket_name, test_folder+'bike_test_rev3.csv')

### SageMaker maintains a separate image for algorithm and region

In [4]:
# Use spot instances

use_spot_instances = True
max_run = 3600
max_wait = 7200

job_name = 'xgboost-bikerental-v1'
checkpoint_s3_uri = None

if use_spot_instances:
    checkpoint_s3_uri = f's3://{bucket_name}/bikerental/checkpoints/{job_name}'
    
#print(f'Checkpoint uri: {checkpoint_s3_uri}')

In [5]:
# EStablish a session with AWS
sess  = sagemaker.Session()

In [6]:
role = get_execution_role()

In [ ]:
# The role has permissions need to train, deploy and execute models
print(role)

In [7]:
# SDK 2 uses image_uris.retrieve the container image location

# Use XGBoost 1.2 version
container = sagemaker.image_uris.retrieve("xgboost",sess.boto_region_name,version="1.2-2")
print(f'Using XGBoost Container {container}')

Using XGBoost Container 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2


### Build Model

In [8]:
# Configure the training job
# Specify type and number of instances to use
# Where final artifacts needs to be stored in S3 location

# for managed spot traing, specify the use_spot_instances flag, mad_run, mad_wait and 
# checkpoint s3 uri

# SDK 2.x version does not require train prefix for instance count and type

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_model_output_location,
    sagemaker_session=sess,
    base_job_name = job_name,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri
)

In [9]:
# Set hyper parameters that is appropriate for training algorithm

estimator.set_hyperparameters(max_depth=5, objective ='reg:squarederror',eta=0.1,num_round=150)

In [10]:
estimator.hyperparameters()

{'max_depth': 5, 'objective': 'reg:squarederror', 'eta': 0.1, 'num_round': 150}

### Speicify Training Data Location and optionally Validation data location

In [11]:
# content type can be libsvm or csv for XGBoost

training_input_config = sagemaker.session.TrainingInput(
    s3_data = s3_training_file_location,
    content_type='csv',
    s3_data_type='S3Prefix')

validation_input_config = sagemaker.session.TrainingInput(
    s3_data = s3_validation_file_location,
    content_type='csv',
    s3_data_type='S3Prefix'
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

In [ ]:
print(training_input_config.config)

In [ ]:
print(validation_input_config.config)

### Training the model

In [13]:
# XGBoost supports 'train' and 'validation' channels

estimator.fit(data_channels)

INFO:sagemaker:Creating training-job with name: xgboost-bikerental-v1-2024-05-06-11-13-48-839


2024-05-06 11:13:49 Starting - Starting the training job...
2024-05-06 11:14:03 Starting - Preparing the instances for training...
2024-05-06 11:14:30 Downloading - Downloading input data...
2024-05-06 11:14:55 Downloading - Downloading the training image...
2024-05-06 11:15:46 Training - Training image download completed. Training in progress.
2024-05-06 11:15:46 Uploading - Uploading generated training model[2024-05-06 11:15:40.507 ip-10-2-76-60.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-05-06:11:15:40:INFO] Imported framework sagemaker_xgboost_container.training
[2024-05-06:11:15:40:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2024-05-06:11:15:40:INFO] No GPUs detected (normal if no gpus installed)
[2024-05-06:11:15:40:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-05-06:11:15:40:INFO] Determined delimiter of CSV input is ','
[2024-05-06:11:15:40:INFO] Determined delimiter of CSV i

In [ ]:
### Deploy Model

predictor = estimator.deploy(initial_instance_count=1,
                            instance_type='ml.m5.xlarge',
                            endpoint_name=job_name)

INFO:sagemaker:Creating model with name: xgboost-bikerental-v1-2024-05-06-11-18-28-783
INFO:sagemaker:Creating endpoint-config with name xgboost-bikerental-v1
INFO:sagemaker:Creating endpoint with name xgboost-bikerental-v1


-----

In [21]:
# SDK 2.0 serializers
from sagemaker.serializers import CSVSerializer

In [22]:
predictor.serializer = CSVSerializer()

In [23]:
predictor.predict([[3,0,1,2,28.7,33.335,79,12.998,2011,7,7,3]])

b'3.773822784423828\n'

In [24]:
import numpy as np

print(np.expm1(3.773822784423828))

42.546214847510505
